# MCP Connectivity Test

Simple notebook to test MCP server connectivity using FastMCP client.  
No LangGraph or LLM required - just direct MCP protocol calls.

## 1. Install/Verify FastMCP

In [ ]:
import importlib.metadata

try:
    version = importlib.metadata.version("fastmcp")
    print(f"fastmcp {version} installed")
except importlib.metadata.PackageNotFoundError:
    print("Installing fastmcp...")
    %pip install fastmcp nest-asyncio -q
    version = importlib.metadata.version("fastmcp")
    print(f"fastmcp {version} installed")

## 2. Imports

In [ ]:
import asyncio
import nest_asyncio

# Enable nested event loops for Jupyter
nest_asyncio.apply()

from fastmcp import Client
from fastmcp.client.transports import StreamableHttpTransport

print("Imports successful!")

## 3. Configuration

Copy credentials from `.mcp-credentials.json`

In [ ]:
# MCP Gateway credentials (from .mcp-credentials.json)
GATEWAY_URL = "YOUR_GATEWAY_URL_HERE"
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN_HERE"

# Validate
if "YOUR_" in GATEWAY_URL or "YOUR_" in ACCESS_TOKEN:
    print("ERROR: Replace GATEWAY_URL and ACCESS_TOKEN")
else:
    print(f"Gateway: {GATEWAY_URL[:60]}...")
    print(f"Token:   {ACCESS_TOKEN[:30]}...")
    print("Configuration OK!")

## 4. Create MCP Client

In [ ]:
transport = StreamableHttpTransport(
    url=GATEWAY_URL,
    headers={"Authorization": f"Bearer {ACCESS_TOKEN}"}
)
client = Client(transport)
print("Client created!")

## 5. List Available Tools

In [ ]:
async def list_tools():
    """List all available tools from the MCP server."""
    async with client:
        return await client.list_tools()

tools = asyncio.get_event_loop().run_until_complete(list_tools())

print(f"Found {len(tools)} tools:")
print("-" * 50)
for tool in tools:
    desc = tool.description or ""
    desc_preview = f"{desc[:60]}..." if len(desc) > 60 else desc
    print(f"  {tool.name}")
    if desc_preview:
        print(f"    {desc_preview}")

## 6. Tool Call Helper

In [ ]:
async def call_tool(name: str, args: dict = None):
    """Call a tool on the MCP server."""
    async with client:
        result = await client.call_tool(name, args or {})
        return result

def run_tool(name: str, args: dict = None):
    """Sync wrapper for calling tools."""
    print(f"Calling: {name}")
    if args:
        print(f"Args: {args}")
    print("-" * 50)
    result = asyncio.get_event_loop().run_until_complete(call_tool(name, args))
    # Handle different result formats
    if hasattr(result, 'content'):
        for item in result.content:
            if hasattr(item, 'text'):
                print(item.text)
            else:
                print(item)
    else:
        print(result)
    return result

print("Helper ready!")

## 7. Test: Get Schema

Adjust `SCHEMA_TOOL` if your server uses a different name.

In [ ]:
SCHEMA_TOOL = "get-schema"  # Adjust if needed

try:
    _ = run_tool(SCHEMA_TOOL)
except Exception as e:
    print(f"Error: {e}")
    print("Check tool name from list above.")

## 8. Test: Run Cypher Query

Adjust `CYPHER_TOOL` if your server uses a different name.

In [ ]:
CYPHER_TOOL = "read-cypher"  # Adjust if needed

query = "MATCH (n) RETURN labels(n) AS label, count(*) AS count LIMIT 10"

try:
    _ = run_tool(CYPHER_TOOL, {"query": query})
except Exception as e:
    print(f"Error: {e}")
    print("Check tool name from list above.")

## 9. Custom Queries

In [ ]:
# Example: run_tool("tool-name", {"arg": "value"})